In [1]:
%load_ext autoreload
%autoreload 2

In [53]:
import os
import argparse
import numpy as np

import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
import networkx as nx

from minimal.arch import Generator

from minimal.dataset import (
    FloorplanGraphDataset,
    floorplan_collate_fn
)

from minimal.utils import (
    init_input,
    draw_masks,
    draw_graph
)

In [3]:
PRETRAINED_PATH = "./checkpoints/pretrained.pth"
DATA_PATH = "./data/sample_list.txt"
OUT_PATH = "./dump"

os.makedirs(OUT_PATH, exist_ok=True)

In [4]:
model = Generator()
model.load_state_dict(torch.load(PRETRAINED_PATH, map_location=torch.device('cpu')), strict=True)
model = model.eval()

In [5]:
# initialize dataset iterator
fp_dataset_test = FloorplanGraphDataset(
    DATA_PATH,
    transforms.Normalize(mean=[0.5], std=[0.5]),
    split='test'
)

fp_loader = torch.utils.data.DataLoader(
    fp_dataset_test, 
    batch_size=1, 
    shuffle=False,
    collate_fn=floorplan_collate_fn
)

6


In [6]:
# run inference
def _infer(graph, model, prev_state=None):
    
    # configure input to the network
    z, given_masks_in, given_nds, given_eds = _init_input(graph, prev_state)
    # run inference model
    with torch.no_grad():
        masks = model(z, given_masks_in, given_nds, given_eds)
        masks = masks.detach().cpu().numpy()
    return masks

In [7]:
i, sample = next(enumerate(fp_loader))

In [15]:
mks, nds, eds, _, _ = sample

In [40]:
real_nodes = np.where(nds.detach().cpu()==1)[-1]

In [41]:
graph = [nds, eds]

In [42]:
true_graph_obj, graph_im = draw_graph([real_nodes, eds.detach().cpu().numpy()])

/home/radium/ser/follows/houseganpp/minimal/utils.py:139: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [57]:
eds.shape

torch.Size([66, 3])